### IMPORTING LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
import os 
import glob
import tqdm
import tensorflow

from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

%matplotlib inline

# from keras.preprocessing.image import load_img,img_to_array

### IMPORT / VIEWING / PREPROCESSING DATASET

In [ ]:
generator = ImageDataGenerator()
batches   = generator.flow_from_directory('../input/100-bird-species/train')

In [ ]:
indices = batches.class_indices

labels  = list(indices.keys())
labels

In [ ]:
for X, y in batches:
    fig, ax = plt.subplots(1, 5, figsize=(20, 20))
    
    for i in range(0,5):
        img = X[i].astype('int')
        label = labels[np.argmax(y[i])]
        ax[i].imshow(img)
        ax[i].set_title(label)
        ax[i].set_xticks([])
        ax[i].set_yticks([])

    
    plt.show()
    break # We only need the first batch

> `DATA AUGMENTATION`

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1/255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   rotation_range=30,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1/255)

test_datagen  = ImageDataGenerator(rescale = 1/255)

In [ ]:
# Reading Directories 

train_directory = '../input/100-bird-species/train'
val_directory   = '../input/100-bird-species/valid'
test_directory  = '../input/100-bird-species/test'

In [ ]:
train_generator=train_datagen.flow_from_directory(train_directory,
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  class_mode='sparse',
                                                  batch_size=256)

In [ ]:
val_generator=valid_datagen.flow_from_directory(val_directory,
                                               target_size=(224,224),
                                               color_mode='rgb',
                                               class_mode='sparse',
                                               batch_size=256)

In [ ]:
test_gemerator=test_datagen.flow_from_directory(test_directory,
                                                target_size=(224,224),
                                                color_mode='rgb',
                                                class_mode='sparse',
                                                batch_size=256)

### BUILDING MODEL (RESTNET101)

In [ ]:
# Model Initialization

from keras.applications import ResNet101V2

convlayer = ResNet101V2(input_shape=(224,224,3),
                        weights='imagenet',
                        include_top=False)

In [ ]:
# Freezing Layers

for layer in convlayer.layers:
    layer.trainable=False

In [ ]:
# Building Model

model=Sequential()
model.add(convlayer)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(225,activation='softmax'))

print(model.summary())

In [ ]:
# Model Compile

opt=tensorflow.keras.optimizers.Adam(lr=0.001)

model.compile(loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              optimizer=opt)

In [ ]:
# Define Callbacks 

filepath = './best_weights.hdf5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 10,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)

callback_list = [earlystopping, checkpoint]

In [ ]:
model_history=model.fit(train_generator,
                        validation_data=val_generator,
                        epochs = 500,
                        callbacks = callback_list,
                        verbose = 1)

REFERENCES 

1) https://www.kaggle.com/anuragmishra2311/birds-classification-using-resnet-101

2) https://www.kaggle.com/naim99/birds-species-classification

3) https://www.kaggle.com/aditya276/bird-cnn-vgg16-99-accuracy-on-test-set